In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf


from keras.callbacks import Callback
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    LearningRateScheduler,
    ReduceLROnPlateau,
    EarlyStopping,
    TensorBoard,
)
from tensorflow.keras import backend as K
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.regularizers import l2
from tensorflow.compat.v2.keras.layers import BatchNormalization
from tensorflow.python.client import device_lib
from utils import *
from models.resnet import *

In [4]:
np.random.seed(1)
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14329798969812477841
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4937233203
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4372473023001701815
physical_device_desc: "device: 0, name: GeForce GTX 1060 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


# Loading images

In [6]:
image_dir_train = "data/training/images/"
files = os.listdir(image_dir_train)
n_train = len(files)
print(f"Loading training images, images loaded: {n_train} ")
imgs_train = np.asarray(
    [load_img(image_dir_train + files[i]) for i in range(n_train)]
)
gt_dir_train = "data/training/groundtruth/"
print(f"Loading groundtruth images, images loaded: {n_train} ")
gt_imgs_train = np.asarray(
    [load_img(gt_dir_train + files[i]) for i in range(n_train)]
)

Loading training images, images loaded: 100 
Loading groundtruth images, images loaded: 100 


In [7]:
imgs_train.shape

(100, 400, 400, 3)

In [8]:
gt_imgs_train.shape

(100, 400, 400)

In [9]:
image_dir_val = "data/validating/images/"
files = os.listdir(image_dir_val)
n_val = len(files)
print(f"Loading validating images, images loaded: {n_val} ")
imgs_val = np.asarray([load_img(image_dir_val + files[i]) for i in range(n_val)])
gt_dir_val = "data/validating/groundtruth/"
print(f"Loading validating groundtruth, images loaded: {n_val} ")
gt_imgs_val = np.asarray([load_img(gt_dir_val + files[i]) for i in range(n_val)])

Loading validating images, images loaded: 20 
Loading validating groundtruth, images loaded: 20 


In [10]:
X_train, Y_train = imag_rotation_aug(imgs_train, gt_imgs_train)

In [11]:
X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)

In [12]:
print(X_train.shape)
print(Y_train.shape)
n_train = Y_train.shape[0]

(900, 448, 448, 3)
(900, 448, 448)


In [13]:
X_val, Y_val = imag_rotation_aug(imgs_val, gt_imgs_val)

In [14]:
X_val = np.asarray(X_val)
Y_val = np.asarray(Y_val)

In [15]:
print(X_val.shape)
print(Y_val.shape)
n_val = Y_val.shape[0]

(180, 448, 448, 3)
(180, 448, 448)


In [16]:
inputs = tf.keras.layers.Input(shape=(64,64,3))
conv_2d = tf.keras.layers.Conv2D(100,64,64)(inputs)
print(type(inputs))
print(type(conv_2d))

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


In [17]:
data_aug_factor = 1
batch_normalization = True
activation = "relu"
dropout = 0
amsgrad = False
lr = 1e-4

In [18]:
# Training parameters
EPOCHS = 80
STEPS_PER_EPOCH = 100
batch_size = 100

model = resnet_unet_model(
    shape=(64, 64, 3),
    batch_normalization=batch_normalization,
    activation=activation,
    dropout=dropout,
    amsgrad=amsgrad,
    lr=lr,
)

# Train the Model


In [19]:
# Train the model
history = model.train(
    EPOCHS, STEPS_PER_EPOCH, n_train, n_val, batch_size, data_aug_factor
)

AttributeError: 'resnet_unet_model' object has no attribute 'train'

In [ ]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure(figsize=(15, 10))
plt.plot(history.history["loss"][1:], label="train_loss")
plt.plot(history.history["val_loss"][1:], label="val_loss")
plt.plot(history.history["accuracy"][1:], label="train_acc")
plt.plot(history.history["val_accuracy"][1:], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("Unet_batchnorm.png")
plt.show()

In [ ]:
# Instantiate the model
lr = 1e-4
model = resnet_unet_model(
    shape=(64, 64, 3),
    batch_normalization=batch_normalization,
    activation=activation,
    dropout=dropout,
    amsgrad=amsgrad,
    lr=lr,
)

# Load the model
model.load("model_batch_relu_80_dropout_0_024_0.804204_0.920300.h5")

# We add all test images to an array, used later for generating a submission
image_filenames = []
for i in range(1, 51):
    image_filename = "data/test_set_images/test_" + str(i) + "/test_" + str(i) + ".png"
    image_filenames.append(image_filename)

# Set-up submission filename
submission_filename = "resnet_unet.csv"

# Generates the submission
generate_submission(model, submission_filename, *image_filenames)